## INTRODUCTION
MindHaven_ChatBot is a domain-specific mental health chatbot designed to simulate counseling conversations using the T5 Transformer model. It helps users navigate emotional difficulties by generating empathetic, informed responses based on a curated dataset from Hugging Face.

In [21]:
# Installing dependencies
!pip install --upgrade transformers datasets sentencepiece


In [22]:
# Importing libraries
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    T5Tokenizer,
    TFT5ForConditionalGeneration,
    DataCollatorForSeq2Seq,
    create_optimizer
)
import tensorflow as tf


In [23]:
# Loading the dataset from Kaggle (updated path)
with open("/kaggle/input/mindhaven-dataset-json/mindhaven_dataset.json", "r") as f:
    data = [json.loads(line) for line in f]

# Converting to DataFrame
df = pd.DataFrame(data)

# Displaying shape and preview
print(f"Total samples: {len(df)}")
df.head()


Total samples: 3512


,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...


## Cleaning & preparing the dataset

In [24]:
# Renaming columns
df.rename(columns={"Context": "input_text", "Response": "target_text"}, inplace=True)

# Dropping NaNs and cleaning text
df.dropna(subset=["input_text", "target_text"], inplace=True)
df["input_text"] = df["input_text"].astype(str).str.strip().str.lower()
df["target_text"] = df["target_text"].astype(str).str.strip()
df = df[(df["input_text"] != "") & (df["target_text"] != "")]

In [25]:
# Splitting into train/val and saving
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
train_df.to_csv("mindhaven_train.csv", index=False)
val_df.to_csv("mindhaven_val.csv", index=False)

In [26]:
# Loading the tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

In [28]:
# Defining tokenization function
def tokenize_data(input_texts, target_texts, tokenizer, max_input_length=128, max_target_length=128):
    input_encodings = tokenizer(
        input_texts,
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
        return_tensors="np"
    )
    target_encodings = tokenizer(
        target_texts,
        padding="max_length",
        truncation=True,
        max_length=max_target_length,
        return_tensors="np"
    )
    return input_encodings, target_encodings


In [29]:
# Tokenizing the clean CSVs
train_df = pd.read_csv("mindhaven_train.csv")
val_df = pd.read_csv("mindhaven_val.csv")

train_inputs, train_targets = tokenize_data(
    train_df["input_text"].tolist(),
    train_df["target_text"].tolist(),
    tokenizer
)

val_inputs, val_targets = tokenize_data(
    val_df["input_text"].tolist(),
    val_df["target_text"].tolist(),
    tokenizer
)

In [30]:
# Converting to tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices({
    "input_ids": train_inputs["input_ids"],
    "attention_mask": train_inputs["attention_mask"],
    "labels": train_targets["input_ids"]
}).shuffle(1000).batch(8).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices({
    "input_ids": val_inputs["input_ids"],
    "attention_mask": val_inputs["attention_mask"],
    "labels": val_targets["input_ids"]
}).batch(8).prefetch(tf.data.AUTOTUNE)


In [31]:
# Loading the t5-small model
model = TFT5ForConditionalGeneration.from_pretrained("t5-small")

# Defining the number of training steps 
EPOCHS = 40
BATCH_SIZE = 8
num_train_steps = len(train_dataset) * EPOCHS

# Optimizer with hyperparameters
optimizer, schedule = create_optimizer(
    init_lr=0.001,             
    num_warmup_steps=200,      
    num_train_steps=num_train_steps
)

# Custom training wrapper
class T5ModelWrapper(tf.keras.Model):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def train_step(self, data):
        x = data
        with tf.GradientTape() as tape:
            outputs = self.model(**x, training=True)
            loss = outputs.loss
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        return {"loss": loss}

    def test_step(self, data):
        x = data
        outputs = self.model(**x, training=False)
        return {"loss": outputs.loss}

# Compiling and training
wrapped_model = T5ModelWrapper(model)
wrapped_model.compile(optimizer=optimizer)

wrapped_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS
)

# Saving the model & tokenizer
model.save_pretrained("./mindhaven_t5_model_tf")
tokenizer.save_pretrained("./mindhaven_t5_model_tf")


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Epoch 1/40


I0000 00:00:1750287386.866950     102 service.cc:148] XLA service 0x7fde9d6788c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750287386.867877     102 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1750287386.979435     102 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1750287387.139975     102 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


395/395 [==============================] - 98s 139ms/step - loss: 3.5759 - val_loss: 3.2875
Epoch 2/40
395/395 [==============================] - 50s 126ms/step - loss: 3.0628 - val_loss: 3.1055
Epoch 3/40
395/395 [==============================] - 50s 126ms/step - loss: 2.8604 - val_loss: 2.9787
Epoch 4/40
395/395 [==============================] - 50s 126ms/step - loss: 2.6908 - val_loss: 2.8890
Epoch 5/40
395/395 [==============================] - 50s 126ms/step - loss: 2.5328 - val_loss: 2.7939
Epoch 6/40
395/395 [==============================] - 50s 126ms/step - loss: 2.3909 - val_loss: 2.7108
Epoch 7/40
395/395 [==============================] - 50s 126ms/step - loss: 2.2575 - val_loss: 2.6458
Epoch 8/40
395/395 [==============================] - 50s 126ms/step - loss: 2.1235 - val_loss: 2.5823
Epoch 9/40
395/395 [==============================] - 50s 126ms/step - loss: 2.0074 - val_loss: 2.5054
Epoch 10/40
395/395 [==============================] - 50s 126ms/step - loss: 1.8909

('./mindhaven_t5_model_tf/tokenizer_config.json',
 './mindhaven_t5_model_tf/special_tokens_map.json',
 './mindhaven_t5_model_tf/spiece.model',
 './mindhaven_t5_model_tf/added_tokens.json')

In [32]:
# Loading the model and tokenizer
from transformers import TFT5ForConditionalGeneration, T5Tokenizer

model = TFT5ForConditionalGeneration.from_pretrained("./mindhaven_t5_model_tf")
tokenizer = T5Tokenizer.from_pretrained("./mindhaven_t5_model_tf")

# Defining the chatbot response function
def ask_bot(question):
    prompt = f"Mental health support: {question.strip()}"

    inputs = tokenizer(
        prompt,
        return_tensors="tf",
        padding="max_length",
        truncation=True,
        max_length=128
    )

    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=100,
        min_length=30,
        no_repeat_ngram_size=3,
        repetition_penalty=1.8,
        num_beams=5,
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

# testing the bot
print("Bot:", ask_bot("I feel anxious and can't sleep at night.what should i do?"))
print("Bot:", ask_bot("I’m always overthinking and feel drained."))
print("Bot:", ask_bot("Lately, I’ve been feeling like I’m not good enough."))

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at ./mindhaven_t5_model_tf.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Bot: Mental health support: You feel anxious and can't sleep at night. What should I do? First off, I would recommend a book called "The Five Agreements" about Mental Health Support. You can read about it here: http://www.psychologytoday.com/what-makes-a-service-for-you-will-bear.htm. Another suggestion is to write a letter to your health care provider about your anxiety and sleep
Bot: Mental Health Support: You are always overthinking and feeling drained. I always suggest that you see a mental health support provider if you have one. There are several reasons why you might feel this way.First, you should check out my website on Psychology Today, which is a blog from the National Institutes of Health. You can search for a support group in your zip code and search more specifically for the National Child Abuse Hotline.
Bot: Mental health support: You're in a place of constant flux and change. I don't know how old you are, but if you are an adult, you might want to have a conversation wi

In [33]:
!zip -r mindhaven_t5_model_tf.zip mindhaven_t5_model_tf


  adding: mindhaven_t5_model_tf/ (stored 0%)
  adding: mindhaven_t5_model_tf/config.json (deflated 63%)
  adding: mindhaven_t5_model_tf/spiece.model (deflated 48%)
  adding: mindhaven_t5_model_tf/added_tokens.json (deflated 83%)
  adding: mindhaven_t5_model_tf/generation_config.json (deflated 29%)
  adding: mindhaven_t5_model_tf/tf_model.h5 (deflated 7%)
  adding: mindhaven_t5_model_tf/tokenizer_config.json (deflated 94%)
  adding: mindhaven_t5_model_tf/special_tokens_map.json (deflated 85%)


In [37]:
import shutil
shutil.move("mindhaven_t5_model_tf.zip", "/kaggle/working/mindhaven_t5_model_tf.zip")


'/kaggle/working/mindhaven_t5_model_tf.zip'

In [38]:
!pip install -q transformers datasets evaluate sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00


In [39]:
# Reloading the trained model & tokenizer 
from transformers import TFT5ForConditionalGeneration, T5Tokenizer
import tensorflow as tf

model = TFT5ForConditionalGeneration.from_pretrained("./mindhaven_t5_model_tf")
tokenizer = T5Tokenizer.from_pretrained("./mindhaven_t5_model_tf")


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at ./mindhaven_t5_model_tf.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [41]:
# Defining the response function

def ask_bot(question):
    prompt = f"Mental health support: {question.strip()}"
    inputs = tokenizer(prompt, return_tensors="tf", padding="max_length", truncation=True, max_length=128)
    output = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=80,
        num_beams=3,
        early_stopping=True
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [42]:
# Performance metrics (BLEU & F1)
import evaluate

# BLEU
bleu = evaluate.load("bleu")

# Sample inputs
prompts = [
    "I feel anxious and can't sleep at night.",
    "I'm feeling lost and alone.",
    "I'm overwhelmed with work and life."
]

# References
references = [
    "Try relaxation exercises or talk to someone you trust.",
    "You're not alone—reach out to a mental health professional.",
    "Take a break, breathe, and try to focus on one step at a time."
]

# Generating predictions
predictions = [ask_bot(q) for q in prompts]

# Computing BLEU
bleu_result = bleu.compute(predictions=predictions, references=[[r] for r in references])

# F1
def compute_token_f1(preds, refs):
    total_f1 = 0.0
    for pred, ref in zip(preds, refs):
        pred_tokens = set(pred.lower().split())
        ref_tokens = set(ref.lower().split())
        common = pred_tokens & ref_tokens
        if not common:
            continue
        precision = len(common) / len(pred_tokens)
        recall = len(common) / len(ref_tokens)
        f1 = 2 * precision * recall / (precision + recall)
        total_f1 += f1
    return total_f1 / len(preds)

f1_score = compute_token_f1(predictions, references)

# results
import pandas as pd
pd.DataFrame({
    "Metric": ["BLEU", "Token-level F1"],
    "Score": [round(bleu_result["bleu"], 4), round(f1_score, 4)]
})


,Metric,Score
0,BLEU,0.0000
1,Token-level F1,0.1276


# Summary

I fine-tuned a T5-small transformer model on domain specific mental health Q&A dataset using TensorFlow and Hugging Face.

Here below are some more


**Details:**
- Model: T5-small
- Epochs: 40
- Learning Rate: 0.001
- Warmup Steps: 200
- Batch Size: 8
- Max Token Length: 128

**Performance Metrics:**
- BLEU Score: **0.0**
- Token-level F1 Score: **0.1276**

**Insights:**
- Although the BLEU score is low, this is expected for generative tasks, the F1 score shows moderate token overlap with references.
- The model generates coherent, domain-relevant, supportive mental health advice.
- Limitations include: small dataset, open-ended nature of responses, and generic expression patterns.
